In [2]:
%pip install albumentations


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\vietv\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.
